## 名称和对象

名称 name 又被称为标识符 identifier；对象 object 具有个体性，多个名称在多个作用域中可以绑定到同一个对象，这在其他语言中称为混叠（aliasing），如语句`a = 1`中，`a`是名称，而`1`是对象，且名称`a`是被绑定在对象`1`上的；在处理不可变的基本类型如数字、字符串、元组时可以忽略这一点，然而对于涉及到列表、字典和大多数其他可变对象，混叠可能对 Python 代码的语义产生很大的影响，如以下代码
```python
a = [1, 2, 3]
b = a
b[0] = 10
print(a, b, sep="\n")
# out:
# [10, 2, 3]
# [10, 2, 3]
```
看似在某些方面产生了不便，但由于混叠在某些方面类似于指针，进而在函数传递对象时较为方便，例如将列表对象作为参数传递给函数时，函数内部对变量（即对象）的修改将直接体现在绑定到该对象的所有名称的取值上，而在 C、C++、Java 等语言中还会涉及到“传引用”的概念

## 命名空间

命名空间是将名称映射到对象的映射的集合。目前大多命名空间均以 Python 字典的形式实现，但这在未来版本中可能会发生变化；不同命名空间之间的名称没有任何关系，若两个模块均定义了一个`maximize`函数，则使用时应在函数前加上模块名加以区分；

命名空间及其生命周期如下

- Python 自带的 builtin 命名空间，如函数`abs()`将名称`abs`映射到对变量取绝对值的操作、`IndexError`将名称`IndexError`映射到索引出错的异常等；该命名空间在 Python 解释器启动时创建，并且一直保留到解释器退出；builtin 命名空间包含了许多 builtin 名称，这些名称是在`builtins`模块中定义的

- 一个模块文件中定义的 global 命名空间，包括类、函数、变量等，该命名空间在模块被读入时创建，通常也会保留到解释器退出

- 函数调用时的 local 命名空间，在调用函数时创建，在函数结束或有函数无法处理的异常被抛出后删除；递归调用函数时，每个函数都有自己的局部命名空间

- 类的实例化对象的属性。以实例化对象`modname`为例，`modname.name`的任何一种`name`均可以使用属性这个词；严格地说，就表达式`modname.name`而言，对`modname`所在模块中定义的名称的引用其实就是对其属性的引用；如此在`modname`所在模块的属性和该模块的 global names 之间便存在一个直接的映射，即这两者位于同一命名空间[1]。属性或是只读的或是可写的，处于后者时可以对属性进行赋值，也可以使用`del`语句删除其可写属性，如`del modname.attrib`将该对象属性中删除`attrib`

对于解释器 top-level invocation 所执行的语句，无论是从脚本文件还是交互读取的，都会被认为是`__main__`模块的一部分，即它们有自己的全局命名空间



## 作用域

作用域是 Python 程序的一个文本区域，该区域内的命名空间的名称可以直接访问；其中 “直接访问” 意味着对某个名称的非限定引用将在此命名空间中查找该名称。作用域是静态确定的，在执行程序的任何时刻，至少有三个嵌套的、其命名空间可以直接访问的作用域：

- 最内层作用域，其包含局部名称，它是在搜索名称时首先被搜索的

- 任何封闭函数的作用域，包含非局部和非全局的名称，在搜索名称时会从最近的封闭作用域开始搜索

- 倒数第二个搜索的作用域，包含当前模块的 global names

- 最后搜索的是最外面的作用域，它是一个包含 builtin names 的命名空间

Python 中只有 module、class、def、lambda才会引入新的作用域，其它的代码块如 if、try、for、while等不引入新作用域，即这些语句内定义的变量在外部也可以访问，如以下代码

```python
for i in range(11):
    pass

print(i)  # out: 10
```

如果一个名称被声明为全局的，那么对其所有的引用和赋值都将直接转到包含模块 global names 的中间作用域中进行；若一个名称想重新绑定最内层作用域以外的变量，可以使用`nonlocal`语句；如果没有声明为`nonlocal`，那么这些变量是只读的，尝试写入这样的变量只会在最内部的作用域中创建一个新的局部变量，而名称相同的外部变量保持不变

通常局部作用域引用当前函数的局部名称；在函数外部，局部作用域会引用与全局作用域相同的命名空间，即模块的命名空间；类的定义会在局部作用域中创建另一个名称空间。

作用域是由上下文确定的，即对于模块中定义的函数，无论以哪种方式调用，其全局作用域即为该模块的命名空间；另一方面对名称的搜索实际是在程序运行时动态完成的；然而语言定义在向着“编译时使用静态名称解析”的方向发展，所以不建议依赖动态名称解析；事实上局部变量已经是静态确定的了

Python 的一个特殊特点是，如果没有全局语句生效，则对名称的赋值总是会在最内部作用域发生；此外赋值并不复制数据，它们只是将名称绑定到对象；对于删除也是如此，即`del x`语句从本地作用域所引用的名称空间中删除对`x`的绑定，实际上，所有引入新名称的操作都是使用局部作用域的，尤其`import`语句和函数定义语句会将模块或函数名称绑定在局部作用域中。

`global`语句可以用来表示特定的变量存在于全局作用域中，并且变量应在那里被重新绑定；`nonlocal`语句表明特定的变量存在于一个封闭的范围内，并且变量应在那里被重新绑定

## 作用域与命名空间的关系

命名空间定义了在某个作用域内变量名和绑定值之间的对应关系，作用域定义了命名空间中的变量能够在多大范围内起作用。




[1]	Except for one thing. Module objects have a secret read-only attribute called __dict__ which returns the dictionary used to implement the module’s namespace; the name __dict__ is an attribute but not a global name. Obviously, using this violates the abstraction of namespace implementation, and should be restricted to things like post-mortem debuggers.

In [ ]:
del spam
def scope_test_1():
    def do_local():
        spam = "local spam"
    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"
    def do_global():
        global spam
        spam = "global spam"
    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)

scope_test_1()
# print(spam)  # will raise an error
print()
spam = "test"
scope_test_1()
print(spam)

In [ ]:
del spam
def scope_test_2():
    def do_local():
        spam = "local spam"
    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"
    def do_global():
        global spam
        spam = "global spam"
    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

spam = 1
scope_test_2()
print(spam)

**解读**：`local`赋值没有改变`scope_test`对`spam`的绑定；`nonlocal`赋值改变了`scope_test`对`spam`的绑定，即将`spam`绑定到了`"nonlocal spam"`上；而`global`赋值改变了模块层级的绑定

In [ ]:
for i in range(12):
    pass

print(i)